In [1]:
import matplotlib as mpl
mpl.use('Agg')
mpl.rcParams['animation.embed_limit'] = 300  # 例: 50MBに増やす
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib import animation
import math
from IPython.display import HTML
import os
from matplotlib.animation import PillowWriter

%matplotlib inline

物理量のSI系と無次元の変換

In [ ]:
# =====================================================
# 1. 物理パラメータ（元コードと同じ形）
# =====================================================

# --- 進行波・電極 ---
Lx     = 0.08          # [m] x方向長さ
waves  = 1.0           # 波数（何波入っているか）
wave_v = 0.5           # [m/s] 波の位相速度
lambda_ = Lx / waves   # [m] 波長
f      = wave_v / lambda_           # [Hz]
k_wave = 2.0 * math.pi / lambda_    # [1/m]
omega  = 2.0 * math.pi * f          # [rad/s]

V0     = 1000.0        # [V] 電極電位振幅
phi0   = 0.0           # [rad] 初期位相
ground_z = 0.0         # [m] 地面高さ

# --- 定数 ---
g      = 1.625         # [m/s^2] 月面重力
eps_0  = 8.85e-12      # [F/m] 真空誘電率

# --- 粒子（代表値）---
R_phys = 3e-5          # [m] 粒子半径（例: 30 µm）
m_phys = 4e-10         # [kg] 粒子質量（例: R=30µmレゴリス相当）
q_phys = 6.5e-14         # [C] 粒子電荷

# --- 摩擦（物理的な係数）---
mu_s_phys = 10        # 静止摩擦係数（例）
mu_k_phys = 10        # 動摩擦係数（例）

# =====================================================
# 2. 無次元スケールと γ, β, Ω の計算
# =====================================================

# 代表電場スケール E0 ≈ k V0
E0 = k_wave * V0   # [V/m]

# 時間スケール T0: クーロン力と慣性のバランスから
#   T0^2 = m / (q k^2 V0)
T0 = math.sqrt(m_phys / (q_phys * (k_wave**2) * V0))

# 無次元パラメータ
gamma_phys = (m_phys * g) / (q_phys * E0)
beta_phys  = (q_phys * k_wave) / (16.0 * math.pi * eps_0 * V0)
Omega_phys = omega * T0



print("=== Physical parameters ===")
print(f"Lx      = {Lx:.3e} m")
print(f"lambda  = {lambda_:.3e} m")
print(f"k_wave  = {k_wave:.3e} 1/m")
print(f"omega   = {omega:.3e} rad/s")
print(f"V0      = {V0:.3e} V")
print(f"R       = {R_phys:.3e} m")
print(f"m       = {m_phys:.3e} kg")
print(f"q       = {q_phys:.3e} C")
print()
print("=== Derived scales ===")
print(f"E0      = {E0:.3e} V/m")
print(f"T0      = {T0:.3e} s  (time scale)")
print()
print("=== Dimensionless parameters from these physical values ===")
print(f"gamma   = {gamma_phys:.3e}   (gravity / electric)")
print(f"beta    = {beta_phys:.3e}   (image / electric)")
print(f"Omega   = {Omega_phys:.3e}   (omega * T0)")

# =====================================================
# 2.5  位置・半径の無次元化（確認用）
# =====================================================

# 長さスケール L0 = 1/k
L0 = 1.0 / k_wave

def x_hat_from_phys(x_phys):
    """物理位置 x[m] を無次元座標 x_hat に変換"""
    return x_phys / L0   # = k_wave * x_phys

def z_hat_from_phys(z_phys):
    return z_phys / L0

def R_hat_from_phys(R_phys_val):
    return R_phys_val / L0

# 例：初期位置 x0 = 0.02 m, z0 = 0.005 m のときの無次元量
x0_phys = 0.02   # [m]
z0_phys = 0.005  # [m]

x0_hat = x_hat_from_phys(x0_phys)
z0_hat = z_hat_from_phys(z0_phys)
R_hat  = R_hat_from_phys(R_phys)
ground_z_hat = z_hat_from_phys(ground_z)

print()
print("=== Example: dimensionless position / radius ===")
print(f"x0_phys = {x0_phys:.5e} m -> x_hat = {x0_hat:.5e}")
print(f"z0_phys = {z0_phys:.5e} m -> z_hat = {z0_hat:.5e}")
print(f"R_phys  = {R_phys:.5e} m -> R_hat = {R_hat:.5e}")
print(f"ground_z = {ground_z:.5e} m -> ground_z_hat = {ground_z_hat:.5e}")


定数

In [ ]:
# =====================================================
# 3. 無次元シミュレーション用パラメータ
# =====================================================

# # # ここから下で、設定
#定数
R_hat = 5e-3    #粒子半径の無次元量
c_hat = R_hat     #地面に接している時、鏡像力の最大値の距離
gamma = 0.7     #重力/最大クーロン力
beta  = 1e-5    #鏡像力/最大クーロン力
Omega = 1.7     #角周波数×時間スケール 波速度/粒子が最大クーロン力から受ける速度
mu_k_hat = 0.2    #動摩擦係数の無次元量
mu_s_hat = 0.5

print()
print("=== Dimensionless parameters actually used in simulation ===")
print(f"gamma(sim) = {gamma:.3e}")
print(f"beta(sim)  = {beta:.3e}")
print(f"Omega(sim) = {Omega:.3e}")
print(f"mu_k_hat(sim)= {mu_k_hat:.3e}")
print(f"mu_s_hat(sim)= {mu_s_hat:.3e}")
print(f"R_hat(sim) = {R_hat:.3e}")
print(f"c_hat(sim) = {c_hat:.3e}")

浮くかどうか

In [ ]:
bbb = beta/c_hat**2
aaa = gamma + bbb
print(bbb)
print(aaa)

摩擦なしでも輸送できるか

Ωはこれより小さい値（初期位置依存）

In [ ]:
x0 = -2.5519  #初期位置
ccc = math.sqrt(2*math.exp(-R_hat)*(1 - math.sin(x0)))
print(ccc)

In [ ]:
print(-2.5519*180/np.pi)

粒子クラス

In [ ]:
class Particle:
    """
    無次元シミュレーション用の粒子。
    x, z, vx, vz, t などはすべて「無次元量」として扱う。
    （物理単位への変換はここでは一切行わない）
    """
    def __init__(self, x0, z0, vx0=0.0, vz0=0.0, R=1.0, label="", color=None):
        # --- 現在状態（すべて無次元量） ---
        self.x  = float(x0)
        self.z  = float(z0)
        self.vx = float(vx0)
        self.vz = float(vz0)
        self.R  = float(R)   # 粒子半径（無次元）

        self.label = label
        self.color = color

        # --- 接触・イベント関係 ---
        self.in_contact = False          # 地面と接触しているか
        self.bounce_count = 0            # 跳ねた回数
        self.collision_times = []        # 衝突した無次元時刻リスト

        # --- 履歴（無次元量）---
        self.hist = {
            "t":  [],
            "x":  [],
            "z":  [],
            "vx": [],
            "vz": [],

            # x方向
            "ax_tot":   [],   # 合力 a_x
            "ax_E":     [],   # 電場（クーロン）による a_x
            "ax_fric":  [],   # 摩擦による a_x（静止 or 動摩擦）
            # （必要なら）他の x 方向成分が出てきたときに拡張

            # z方向
            "az_tot":   [],   # 合力 a_z
            "az_E":     [],   # 電場（クーロン）による a_z
            "az_img":   [],   # 鏡像力による a_z
            "az_g":     [],   # 重力による a_z

            # 接触情報
            "N_hat":        [],   # 法線反力（無次元）
            "in_contact":   [],   # 地面と接触しているか（True/False を 0/1 でも可）

        }

    def save(self,
             t_hat,
             ax_tot=0.0, az_tot=0.0,
             ax_E=0.0,   az_E=0.0,
             az_img=0.0, az_g=0.0,
             ax_fric=0.0,
             N_hat=0.0,
             in_contact=False):
        """
        無次元時間 t における現在状態を履歴に保存
        """
        self.hist["t"].append(float(t_hat))
        self.hist["x"].append(self.x)
        self.hist["z"].append(self.z)
        self.hist["vx"].append(self.vx)
        self.hist["vz"].append(self.vz)

        # 加速度・接触情報
        self.hist["ax_tot"].append(ax_tot)
        self.hist["az_tot"].append(az_tot)
        self.hist["ax_E"].append(ax_E)
        self.hist["az_E"].append(az_E)
        self.hist["az_img"].append(az_img)
        self.hist["az_g"].append(az_g)
        self.hist["ax_fric"].append(ax_fric)
        self.hist["N_hat"].append(N_hat)
        self.hist["in_contact"].append(1 if in_contact else 0)

# -------------------------------------------------
    # 物理量から直接 Particle を作るクラスメソッド
    # -------------------------------------------------
    @classmethod
    def from_physical(cls,
                      x0_phys, z0_phys,
                      vx0=0.0, vz0=0.0,
                      R_phys_val=R_phys,
                      label="",
                      color=None):
        """
        物理座標 (x0_phys [m], z0_phys [m], R_phys [m]) から
        無次元粒子を生成するヘルパー。
        速度 vx0, vz0 は「すでに無次元化された値」として与える前提。
        （もし速度も物理単位から無次元化したくなったら、そのとき専用関数を追加）
        """
        x_hat0 = x_hat_from_phys(x0_phys)
        z_hat0 = z_hat_from_phys(z0_phys)
        R_hat  = R_hat_from_phys(R_phys_val)

        return cls(
            x0=x_hat0,
            z0=z_hat0,
            vx0=vx0,
            vz0=vz0,
            R=R_hat,
            label=label,
            color=color,
        )

関数

In [ ]:
# =====================================================
# 5. 無次元運動方程式：各力の項
# =====================================================
### SI単位の物理量使う時
# EPS_CONTACT_phys = max(1e-10, 1e-5 * R_phys)  # 物理単位での接触判定用しきい値
# EPS_CONTACT = EPS_CONTACT_phys / L0  # 無次元化したもの
# VEL_EPS_phys = 1e-9 # [m/s] 物理単位での速度ゼロ判定用しきい値
# VEL_EPS = VEL_EPS_phys / (L0 / T0)  # 無次元化したもの

### 無次元量使う時
contact_rel = 1e-4
EPS_CONTACT = contact_rel * R_hat

VEL_EPS = 1e-6

def electric_accel(p: Particle, t: float):
    """
    クーロン力（進行波電場）による無次元加速度項 (a_x, a_z) を返す。
    t は無次元時間。
    """
    phase = p.x - Omega * t + phi0
    exp_factor = math.exp(-p.z)

    ax = -exp_factor * math.cos(phase)
    az =  exp_factor * math.sin(phase)
    return ax, az

def image_accel(p: Particle):
    """
    鏡像力による無次元加速度項 (a_x, a_z) を返す。
    x方向成分は 0、z方向のみ。
    """
    # z が 0 に近づきすぎると 1/z^2 が発散するので、最小値を入れておく
    z_eff = (p.z - p.R) + c_hat - ground_z_hat

    s_min = 0.1 * p.R  # とりあえず 0.1R など
    if z_eff < s_min:
        z_eff = s_min

    az = -beta / (z_eff**2)
    return 0.0, az

def gravity_accel():
    """
    重力による無次元加速度項 (a_x, a_z) を返す。
    x 方向は 0、z 方向のみ一定。
    """
    az = -gamma
    return 0.0, az


def contact_and_friction_accel(p: Particle, t: float):
    """
    接触判定＋静止摩擦／動摩擦込みの加速度を計算する。
    戻り値:
        ax_total, az_total, in_contact, N_hat
    """
    # まず「電場＋鏡像力＋重力」の項を計算
    ax_e, az_e = electric_accel(p, t)
    _,   az_img = image_accel(p)
    _,   az_g   = gravity_accel()

    az_free = az_e + az_img + az_g

    # 接触判定（粒子下端が地面を下回っていれば接触とみなす）
    in_contact = ((p.z - p.R) <= (ground_z_hat + EPS_CONTACT))

    N_hat = 0.0
    az_total = az_free

    # --- 法線反力の計算 ---
    if in_contact and az_free < 0.0:
        # 下向きに加速しようとしているので、床が押し返す
        # z方向加速度を 0 にするように反力を入れる
        N_hat = -az_free
        az_total = 0.0
    else:
        # それ以外は反力なし（浮いている or 上向き加速）
        N_hat = 0.0
        az_total = az_free
        # 浮き上がるときは接触フラグを落とすことも考えられる
        if in_contact and az_free >= 0.0:
            in_contact = False

    # --- x方向：摩擦 ---
    ax_fric  = 0.0        # まず摩擦は0としておく
    ax_total = ax_e

    if in_contact:
        # 静止摩擦条件のチェック
        # 1. 速度がほぼゼロ
        # 2. クーロン力による加速度が静止摩擦限界以下
        if (abs(p.vx) < VEL_EPS) and (abs(ax_e) <= mu_s_hat * N_hat):
            # 静止摩擦：滑り出さないように完全に打ち消す
            ax_fric  = -ax_e      # 電場を打ち消す向き・大きさ
            ax_total = ax_e + ax_fric   # = 0
        else:
            # 動摩擦：速度方向と逆向きに mu_k * N_hat を加える
            if abs(p.vx) > VEL_EPS:
                sign = math.copysign(1.0, p.vx)
            else:
                # 速度がほぼ 0 なら、電場による加速方向に滑り出すと仮定
                sign = math.copysign(1.0, ax_e) if ax_e != 0.0 else 0.0

            ax_fric  = -mu_k_hat * N_hat * sign
            ax_total = ax_e + ax_fric
            # ax_total = 0.0  #摩擦力大きくする時

    return ax_total, az_total, in_contact, N_hat, ax_fric


def accel_total(p: Particle, t: float):
    """
    粒子 p の無次元加速度 (ax, az) を返す高水準関数。
    - クーロン力
    - 鏡像力
    - 重力
    - 接触・静止摩擦・動摩擦
    をすべて含めたもの。
    """
    ax, az, in_contact, N_hat, ax_fric = contact_and_friction_accel(p, t)

    # 接触状態を粒子にも反映しておく（イベント判定などに使える）
    p.in_contact = in_contact
    return ax, az


# =====================================================
# RK4 1ステップ分
# =====================================================

def rk4_step(p: Particle, t: float, dt: float):
    """
    粒子 p を、無次元時間 t から t+dt まで RK4 で1ステップ進める。
    p の状態 (x, z, vx, vz) を in-place で更新する。
    """

    # 現在状態を退避しておく
    x0, z0 = p.x, p.z
    vx0, vz0 = p.vx, p.vz
    in_contact0 = p.in_contact

    # accel_total を「任意の状態 (x,z,vx,vz,t) に対して」使うためのヘルパ
    def deriv(x, z, vx, vz, t_local):
        """
        状態 (x,z,vx,vz) における微分:
            dx/dt = vx
            dz/dt = vz
            dvx/dt, dvz/dt = accel_total(...)
        を返す。
        """
        # p を一時的にこの状態にして accel_total を呼ぶ
        p.x, p.z, p.vx, p.vz = x, z, vx, vz
        ax, az = accel_total(p, t_local)

        # 状態を元に戻す（p.in_contact も戻す）
        p.x, p.z, p.vx, p.vz = x0, z0, vx0, vz0
        p.in_contact = in_contact0

        return vx, vz, ax, az

    # k1
    k1_x, k1_z, k1_vx, k1_vz = deriv(x0, z0, vx0, vz0, t)

    # k2
    x2  = x0  + 0.5 * dt * k1_x
    z2  = z0  + 0.5 * dt * k1_z
    vx2 = vx0 + 0.5 * dt * k1_vx
    vz2 = vz0 + 0.5 * dt * k1_vz
    k2_x, k2_z, k2_vx, k2_vz = deriv(x2, z2, vx2, vz2, t + 0.5 * dt)

    # k3
    x3  = x0  + 0.5 * dt * k2_x
    z3  = z0  + 0.5 * dt * k2_z
    vx3 = vx0 + 0.5 * dt * k2_vx
    vz3 = vz0 + 0.5 * dt * k2_vz
    k3_x, k3_z, k3_vx, k3_vz = deriv(x3, z3, vx3, vz3, t + 0.5 * dt)

    # k4
    x4  = x0  + dt * k3_x
    z4  = z0  + dt * k3_z
    vx4 = vx0 + dt * k3_vx
    vz4 = vz0 + dt * k3_vz
    k4_x, k4_z, k4_vx, k4_vz = deriv(x4, z4, vx4, vz4, t + dt)

    # RK4 更新
    p.x  = x0  + (dt / 6.0) * (k1_x  + 2.0 * k2_x  + 2.0 * k3_x  + k4_x)
    p.z  = z0  + (dt / 6.0) * (k1_z  + 2.0 * k2_z  + 2.0 * k3_z  + k4_z)
    p.vx = vx0 + (dt / 6.0) * (k1_vx + 2.0 * k2_vx + 2.0 * k3_vx + k4_vx)
    p.vz = vz0 + (dt / 6.0) * (k1_vz + 2.0 * k2_vz + 2.0 * k3_vz + k4_vz)

    # 最後に、この新しい状態に対して in_contact を更新しておく
    now_in_contact = (p.z - p.R) <= (ground_z_hat + EPS_CONTACT)
    p.in_contact = now_in_contact



def t_end_hat_from_cycles(N_cycles, Omega):
    """
    N_cycles：何周期見るか
    Omega：無次元角周波数
    """
    T_hat = 2.0 * math.pi / Omega         # 無次元1周期
    t_end_hat = N_cycles * T_hat          # 無次元時間の合計
    return t_end_hat



def choose_dt_hat_for_Omega(Omega: float):
    """
    Ω に応じて、無次元時間ステップ dt_far/mid/near を決めるヘルパー。
    目安:
      - 遠方:   1周期あたり ~600ステップ
      - 中間:   ~6000ステップ
      - 近接:  ~60000ステップ
    """

    # 安全側の係数（必要なら後で変えてOK）
    fac_far  = 1e-2   # 0.01 / Ω
    fac_mid  = 1e-3   # 0.001 / Ω
    fac_near = 1e-4   # 0.00001 / Ω

    dt_far_hat  = fac_far  / Omega
    dt_mid_hat  = fac_mid  / Omega
    dt_near_hat = fac_near / Omega

    # 履歴保存間隔
    save_dt_hat = 5*dt_near_hat

    return dt_far_hat, dt_mid_hat, dt_near_hat, save_dt_hat


def simulate(
    p: Particle,
    t_end_hat: float,
    dt_far_hat: float,
    dt_mid_hat: float,
    dt_near_hat: float,
    save_dt_hat: float | None = None,
    ):
    """
    物理時間 [s] ベースの設定（t_end, dt_far, dt_mid, dt_near）を引数に取り、
    内部で無次元時間に変換してシミュレーションする。
    """

    t_hat  = 0.0            # 無次元時間

    # 初期接触状態
    in_contact = (p.z - p.R) <= (ground_z_hat + EPS_CONTACT)
    last_bounce_time = -1e9          # あり得ないくらい小さい値で初期化
    min_bounce_interval = 1        # これが「同一衝突とみなす時間幅」
    p.in_contact = in_contact
    prev_in_contact = in_contact

    # --- 初期状態の加速度を計算して保存 ---
    ax_E0, az_E0 = electric_accel(p, t_hat)
    _,   az_img0 = image_accel(p)
    _,   az_g0   = gravity_accel()
    ax_tot0, az_tot0, in_contact0, N_hat0, ax_fric0 = contact_and_friction_accel(p, t_hat)


    # 初期状態を保存
    p.save(
        t_hat,
        ax_tot=ax_tot0,
        az_tot=az_tot0,
        ax_E=ax_E0,
        az_E=az_E0,
        ax_fric=ax_fric0,
        az_img=az_img0,
        az_g=az_g0,
        N_hat=N_hat0,
        in_contact=in_contact0,
    )
    events = []
    time_since_save_hat = 0.0

    while t_hat < t_end_hat:
        # ---- 元コードと同じロジックで dt_hat を決める ----
        # ここは z, R, EPS_CONTACT を無次元 → 物理に戻してもいいし、
        # 無次元のまま「EPS_CONTACT も無次元」にして比較してもいい。
        if (p.z - p.R) > 50.0 * EPS_CONTACT:
            dt_hat = dt_far_hat
        elif (p.z - p.R) > 10.0 * EPS_CONTACT:
            dt_hat = dt_mid_hat
        else:
            dt_hat = dt_near_hat

        if t_hat + dt_hat > t_end_hat:
            dt_hat = t_end_hat - t_hat

        vz_before = p.vz

        # 1ステップ進める（無次元時間で）
        rk4_step(p, t_hat, dt_hat)
        t_hat  += dt_hat

        # 地面より下にめり込んだ場合の補正などは今の simulate と同じでOK
        if (p.z - p.R) < ground_z_hat:
            p.z = ground_z_hat + p.R
            p.vz = 0.0
            # p.vx = 0.0  #摩擦力大きくする時

        now_in_contact = (p.z - p.R) <= (ground_z_hat + EPS_CONTACT)
        p.in_contact = now_in_contact

        if (not prev_in_contact) and now_in_contact:
            if (vz_before < 0.0) and (t_hat - last_bounce_time > min_bounce_interval):
                p.bounce_count += 1
                p.collision_times.append(t_hat)  # 物理時間で記録してもいい
                events.append({"time": t_hat, "type": "bounce"})
                last_bounce_time = t_hat   # ここを必ず更新！

        if prev_in_contact and (not now_in_contact):
            events.append({"time": t_hat, "type": "lift-off"})

        prev_in_contact = now_in_contact

        # 履歴保存（ここでは無次元時間で保存している）
        time_since_save_hat += dt_hat
        if (save_dt_hat is None) or (time_since_save_hat >= save_dt_hat):
            # 現在状態で各加速度成分を計算
            ax_E, az_E = electric_accel(p, t_hat)
            _,   az_img = image_accel(p)
            _,   az_g   = gravity_accel()
            ax_tot, az_tot, in_contact_cf, N_hat, ax_fric = contact_and_friction_accel(p, t_hat)

            p.save(
                t_hat,
                ax_tot=ax_tot,
                az_tot=az_tot,
                ax_E=ax_E,
                az_E=az_E,
                ax_fric=ax_fric,
                az_img=az_img,
                az_g=az_g,
                N_hat=N_hat,
                in_contact=in_contact_cf,
            )
            
            time_since_save_hat = 0.0

    return p.hist, events


def compute_energy(hist_nd, Omega, R_hat, phi0=0.0):
    """
    無次元履歴 hist_nd から
      ψ, ψ', V(ψ), H = 1/2 ψ'^2 + A(1 - sinψ)
    を計算して返す。
    """
    t_hat = np.array(hist_nd["t"])
    x_hat = np.array(hist_nd["x"])
    vx_hat = np.array(hist_nd["vx"])

    # 位相差 ψ, 速度差 ψ' = v_x - Ω
    psi     = x_hat - Omega * t_hat + phi0
    psi_dot = vx_hat - Omega

    # 振幅 A ≃ e^{-R̂}（z≃R̂ に固定した近似）
    A = np.exp(-R_hat)

    # 有効ポテンシャル（定数シフト 1 は見た目のためだけで物理的には不要）
    # V = A * (1.0 - np.sin(psi))
    V = A * np.sin(psi)

    # 有効エネルギー
    H = 0.5 * psi_dot**2 + V

    return t_hat, psi, psi_dot, V, H


何周期見るか

In [ ]:
# n周期ぶん見る
N_cycles = 20
t_end_hat = t_end_hat_from_cycles(N_cycles, Omega)

# Ω から dt_far/mid/near を決める
dt_far_hat, dt_mid_hat, dt_near_hat, save_dt_hat = choose_dt_hat_for_Omega(Omega)

摩擦係数を変えて複数粒子をシミュレーション

In [ ]:
mu_pairs = [
    # (0,0),
    (0.2, 0.0),
    (0.2, 0.1),
    (0.2, 0.2),
    # (0.4, 0.3),
    # (0.5, 0.1),
    # (0.5, 0.2),
    # (0.5, 0.3),
    # (1.0, 0.2),
]

results_by_mu = []

for mu_s_hat, mu_k_hat in mu_pairs:
    print(f"Simulating with mu_s={mu_s_hat}, mu_k={mu_k_hat}")

    # グローバルを書き換え
    globals()["mu_s_hat"] = mu_s_hat
    globals()["mu_k_hat"] = mu_k_hat

    # 粒子を毎回リセット（同じ初期条件）
    p = Particle(
        # x0=-2.5519,
        # x0 = 45*np.pi/180,
        x0 = np.pi/2,
        # x0 = 0.0,
        z0=R_hat,   # 接地
        vx0=0.0,
        vz0=0.0,
        R=R_hat,
        label=f"Ω={Omega}, μs={mu_s_hat}, μk={mu_k_hat}",
        color=None,
    )

    hist_nd, events = simulate(
        p,
        t_end_hat=t_end_hat,
        dt_far_hat=dt_far_hat,
        dt_mid_hat=dt_mid_hat,
        dt_near_hat=dt_near_hat,
        save_dt_hat=save_dt_hat,
    )

    results_by_mu.append({
        "mu_s": mu_s_hat,
        "mu_k": mu_k_hat,
        "p": p,
        "hist_nd": hist_nd,
        "events": events,
    })


---------------------------

位置変化の可視化

In [ ]:
# --------------------------------------------------------
# 1枚目：x_hat(t)
# --------------------------------------------------------
plt.figure(figsize=(8, 5))

for res in results_by_mu:
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat = np.array(hist["t"])
    x_hat = np.array(hist["x"])

    label = f"μs={mu_s}, μk={mu_k}"
    plt.plot(t_hat, x_hat, label=label)

# wave position
t_hat_ref = np.linspace(0, t_hat.max(), 1000)
x_wave = Omega * t_hat_ref
plt.plot(t_hat_ref, x_wave, "--", color="k", label="wave (Ω t)")

plt.xlabel("t_hat")
plt.ylabel("x_hat")
# plt.title("Position x_hat(t) for each (μs, μk)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



# --------------------------------------------------------
# 2枚目：z_hat(t)
# --------------------------------------------------------
plt.figure(figsize=(10, 5))

for res in results_by_mu:
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat = np.array(hist["t"])
    z_hat = np.array(hist["z"])

    label = f"μs={mu_s}, μk={mu_k}"
    plt.plot(t_hat, z_hat, label=label)

# contact height
plt.axhline(R_hat, linestyle="--", color="gray", label="contact height")

plt.xlabel("t_hat")
plt.ylabel("z_hat")
plt.title("Position z_hat(t) for each (μs, μk)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


位相差

In [ ]:
plt.figure(figsize=(10, 6))

for res in results_by_mu:
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat = np.array(hist["t"])
    x_hat = np.array(hist["x"])

    # --- 位相 ψ = x_hat - Ω t ---
    psi = x_hat - Omega * t_hat   # φ0 = 0 前提

    label = f"μs={mu_s}, μk={mu_k}"
    plt.plot(t_hat, psi, label=label)

plt.xlabel("t_hat")
plt.ylabel("psi = x_hat - Ω t")
plt.title("Phase ψ(t) for each (μs, μk)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))

# 色をここで指定（例：μkが小さい方→緑、大きい方→赤）
# colors = ["blue","orange", "green"]

for idx, res in enumerate(results_by_mu):
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat = np.array(hist["t"])
    x_hat = np.array(hist["x"])

    # --- 位相 ψ = x_hat - Ω t ---
    psi = x_hat - Omega * t_hat
    # psi_mod = (psi + np.pi) % (2*np.pi) - np.pi  # [-π, π] に折り畳み
    psi_mod = psi % (2*np.pi)

    label = f"μs={mu_s}, μk={mu_k}"
    plt.plot(t_hat, psi_mod, label=label) #, color=colors[idx])

plt.xlabel("t_hat")
plt.ylabel("ψ_mod")
plt.title("Phase ψ_mod(t) for each (μs, μk)")

# tick_vals = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi]
# tick_labels = ["-π", "-π/2", "0", "π/2", "π"]
tick_vals = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
tick_labels = ["0", "π/2", "π", "3π/2", "2π"]
plt.yticks(tick_vals, tick_labels)

plt.axhline(0.0,      color="gray", linestyle="--", linewidth=1)
plt.axhline(np.pi/2,  color="gray", linestyle="--", linewidth=1)
plt.axhline(-np.pi/2, color="gray", linestyle="--", linewidth=1)

plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


加速度-時間変化

In [ ]:
# ========================================================
# 加速度 & 垂直抗力 vs 時間  （摩擦係数ごとに図を分ける）
# ========================================================
for res in results_by_mu:
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat   = np.array(hist["t"])
    ax_tot  = np.array(hist["ax_tot"])
    ax_E    = np.array(hist["ax_E"])
    ax_fric = np.array(hist["ax_fric"])
    N_hat   = np.array(hist["N_hat"])

    fig, ax0 = plt.subplots(figsize=(8, 5))
    # fig.suptitle(f"Ω={Omega:.3f}, μs={mu_s}, μk={mu_k}", fontsize=14)

    ax0.plot(t_hat, ax_E,    label="a_x (E)",     linewidth=1)
    ax0.plot(t_hat, ax_fric, label="a_x (fric)",  linewidth=1)
    ax0.plot(t_hat, ax_tot,  label="a_x (total)", linewidth=1.5)
    # ax0.plot(t_hat, N_hat,   label="N_hat",       linewidth=1)

    ax0.set_xlabel("t_hat")
    ax0.set_ylabel("value")
    ax0.set_title(f"x-acceleration (μs={mu_s}, μk={mu_k})")
    ax0.grid(True)
    ax0.legend()

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


加速度-位相

In [ ]:
# ========================================================
# 加速度 & 垂直抗力 vs 位相  （摩擦係数ごとに図を分ける：1枚にまとめ版）
# ========================================================
for res in results_by_mu:
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat   = np.array(hist["t"])
    x_hat   = np.array(hist["x"])
    ax_tot  = np.array(hist["ax_tot"])
    ax_E    = np.array(hist["ax_E"])
    ax_fric = np.array(hist["ax_fric"])
    N_hat   = np.array(hist["N_hat"])

    # --- 位相 ψ, ψ_mod ---
    psi     = x_hat - Omega * t_hat        # φ0 = 0 と仮定
    psi_mod = np.mod(psi, 2.0 * np.pi)     # 0〜2π に折り畳み
    # psi_mod = psi

    # ここを 1 枚の図に変更
    fig, ax0 = plt.subplots(figsize=(10, 6))
    fig.suptitle(f"Ω={Omega:.3f}, μs={mu_s}, μk={mu_k}", fontsize=14)

    # --- 位相 vs 各量を 1 枚にまとめて描画 ---
    ax0.scatter(psi_mod, ax_E,    s=4, alpha=0.4, label="a_x (E)")
    ax0.scatter(psi_mod, ax_fric, s=4, alpha=0.4, label="a_x (fric)")
    ax0.scatter(psi_mod, ax_tot,  s=4, alpha=0.4, label="a_x (total)")
    ax0.scatter(psi_mod, N_hat,   s=4, alpha=0.4, label="N_hat")

    ax0.set_xlabel("ψ_mod = (x_hat - Ω t_hat) mod 2π")
    ax0.set_ylabel("value (a_x_hat, N_hat)")
    ax0.set_title("x-acceleration & normal force vs phase ψ")
    ax0.grid(True)
    ax0.legend(markerscale=3)

    # x 軸の目盛りを 0, π/2, π, ... にしておく
    tick_vals   = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
    tick_labels = ["0", "π/2", "π", "3π/2", "2π"]
    ax0.set_xticks(tick_vals)
    ax0.set_xticklabels(tick_labels)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
# ========================================================
# 加速度 & 垂直抗力 vs 位相（unwrap）  摩擦係数ごとに図を分ける
# ========================================================
for res in results_by_mu:
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat   = np.array(hist["t"])
    x_hat   = np.array(hist["x"])
    ax_tot  = np.array(hist["ax_tot"])
    ax_E    = np.array(hist["ax_E"])
    ax_fric = np.array(hist["ax_fric"])
    N_hat   = np.array(hist["N_hat"])

    # --- 位相 ψ を計算し、2πで wrap しないように「ほどく」 ---
    psi = x_hat - Omega * t_hat        # φ0 = 0 と仮定
    psi_unwrap = np.unwrap(psi)        # 2πジャンプをつなげて単調増加にする

    # 「何周期目か」を横軸にしても分かりやすいので 2π で割っておく
    # phase_cycle = psi_unwrap / (2.0 * np.pi)  # 0,1,2,... が各周期
    phase_cycle = np.mod(psi, 2.0 * np.pi) 

    fig, ax0 = plt.subplots(figsize=(10, 6))
    fig.suptitle(f"Ω={Omega:.3f}, μs={mu_s}, μk={mu_k}", fontsize=14)

    # --- 1枚に全部の線を描く ---
    ax0.plot(phase_cycle, ax_E,    label="a_x (E)",     linewidth=2)
    ax0.plot(phase_cycle, ax_fric, label="a_x (fric)",  linewidth=1)
    ax0.plot(phase_cycle, ax_tot,  label="a_x (total)", linewidth=1.5)
    ax0.plot(phase_cycle, N_hat,   label="N_hat",       linewidth=1)

    ax0.set_xlabel("phase cycles  ψ / 2π  (時間とともに単調増加)")
    ax0.set_ylabel("value (a_x_hat, N_hat)")
    ax0.set_title("x-acceleration & normal force vs phase (unwrapped)")
    ax0.grid(True)
    ax0.legend()

    plt.tight_layout()
    plt.show()


相対速度-時間

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# 加速度グラフと被らない色（5個）
colors = ["brown","green", "red"]

for idx, res in enumerate(results_by_mu):
    mu_s = res["mu_s"]
    mu_k = res["mu_k"]
    hist = res["hist_nd"]

    t_hat = np.array(hist["t"])
    vx_hat = np.array(hist["vx"])

    # 相対速度
    v_rel = vx_hat - Omega

    label = f"μs={mu_s}, μk={mu_k}"
    plt.plot(t_hat, v_rel, label=label) #, color=colors[idx])

plt.axhline(0.0, linestyle="--", color="gray")
plt.xlabel("t_hat")
plt.ylabel("v_rel = vx_hat - Ω")
plt.title("Relative velocity v_rel(t) for each (μs, μk)")

plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


相対速度-位相差

In [ ]:
def plot_vrel_phase_scatter(Omega, phi0, unwrap=False):
    for entry in results_by_mu:
        mu_s = entry["mu_s"]
        mu_k = entry["mu_k"]
        hist_nd = entry["hist_nd"]

        t_hat  = np.array(hist_nd["t"])
        x_hat  = np.array(hist_nd["x"])
        vx_hat = np.array(hist_nd["vx"])

        psi = x_hat - Omega * t_hat + phi0

        # unwrap or wrap
        if unwrap:
            psi_plot = np.unwrap(psi)
            xlabel = r"unwrapped phase $\psi$ [rad]"
        else:
            psi_plot = np.mod(psi, 2.0*np.pi)
            xlabel = r"phase $\psi$ mod $2\pi$ [rad]"

        v_rel = vx_hat - Omega

        # ---- プロット ----
        fig, ax = plt.subplots(figsize=(10, 5))
        fig.suptitle(f"Ω={Omega:.3f}, μs={mu_s}, μk={mu_k}")

        sc = ax.scatter(psi_plot, v_rel, c=t_hat, s=8,
                        cmap="viridis", alpha=0.7)

        ax.axhline(0, color="gray", linewidth=1)

        # ----- ★ unwrap = True のとき π刻み目盛りを付ける -----
        if unwrap:
            psi_min = psi_plot.min()
            psi_max = psi_plot.max()

            # πごとの目盛り値
            n_min = int(np.floor(psi_min / np.pi))
            n_max = int(np.ceil(psi_max / np.pi))
            tick_vals = [n*np.pi for n in range(n_min, n_max+1)]
            tick_labels = [f"{n}π" for n in range(n_min, n_max+1)]

            ax.set_xticks(tick_vals)
            ax.set_xticklabels(tick_labels)

        else:
            # 通常の [0, π/2, π, …]
            tick_vals   = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
            tick_labels = ["0", "π/2", "π", "3π/2", "2π"]
            ax.set_xticks(tick_vals)
            ax.set_xticklabels(tick_labels)

        ax.set_xlabel(xlabel)
        ax.set_ylabel(r"$v_{\rm rel} = \hat v_x - \Omega$")
        ax.grid(True)

        cbar = plt.colorbar(sc, ax=ax)
        cbar.set_label("t_hat")

        plt.tight_layout()
        plt.show()


In [ ]:
plot_vrel_phase_scatter(Omega, phi0, unwrap=True)

-------------------------

アニメーション時間変化

In [ ]:
# # =========================================
# # 0. データの取り出し ＆ 共通時間軸の作成
# # =========================================
# t_lists = []
# x_lists = []
# z_lists = []
# labels  = []

# for entry in results_by_mu:
#     hist_nd = entry["hist_nd"]
#     t_arr = np.array(hist_nd["t"])
#     x_arr = np.array(hist_nd["x"])
#     z_arr = np.array(hist_nd["z"])

#     t_lists.append(t_arr)
#     x_lists.append(x_arr)
#     z_lists.append(z_arr)
#     labels.append(entry["p"].label)

# # 最小長さに揃える
# min_len = min(len(t) for t in t_lists)
# t_base  = t_lists[0][:min_len]
# x_lists = [x[:min_len] for x in x_lists]
# z_lists = [z[:min_len] for z in z_lists]

# # =========================================
# # 1. フレーム間引き（容量対策）
# # =========================================
# target_frames = 600           # だいたいこれくらいに収める
# step = max(1, len(t_base) // target_frames)

# indices = np.arange(0, len(t_base), step)
# t_anim  = t_base[indices]

# x_anim_lists = [x[indices] for x in x_lists]
# z_anim_lists = [z[indices] for z in z_lists]

# # =========================================
# # 2. x は 1 波長 [0, 2π) の範囲だけ描く
# #    → 出ていった粒子は周期境界で戻す
# # =========================================
# Lx_hat_period = 2.0 * np.pi
# x_min = 0.0
# x_max = Lx_hat_period

# def wrap_x_hat(x):
#     """x を [0, 2π) に折りたたむ"""
#     return np.mod(x, Lx_hat_period)

# # z 範囲：地面から少し上＋減衰がわかるように上側を広めに
# all_z = np.concatenate(z_anim_lists)
# z_min = ground_z_hat
# z_data_max = all_z.max()
# margin_z = 0.1 * max(z_data_max - z_min, 1.0)
# z_max = max(z_data_max + margin_z, 3.0)   # exp(-z) の減衰が見えるように最低 3 くらいまで



# # =========================================
# # 3. 電位 φ̂(x,z,t) のグリッド
# # =========================================
# nx, nz = 400, 200            # 解像度はそのまま高め維持
# x_grid = np.linspace(x_min, x_max, nx)
# z_grid = np.linspace(z_min, z_max, nz)
# X, Z   = np.meshgrid(x_grid, z_grid)

# def phi_hat_field(t_hat_val: float) -> np.ndarray:
#     """
#     φ̂(x,z,t) = exp(-z) * sin(x - Ω t + φ0)
#     粒子の運動で使っているのと同じポテンシャル
#     """
#     phase = X - Omega * t_hat_val + phi0
#     # return np.exp(-Z) * np.sin(phase) #電位
#     return -np.exp(-Z) * np.cos(phase)  #加速度

# # =========================================
# # 4. 図とアーティストの準備
# # =========================================
# fig, ax = plt.subplots(figsize=(7, 4))

# Phi0 = phi_hat_field(t_anim[0])

# im = ax.imshow(
#     Phi0,
#     extent=[x_min, x_max, z_min, z_max],
#     origin="lower",
#     aspect="auto",
#     cmap="RdBu_r",
# )

# cbar = fig.colorbar(im, ax=ax)
# cbar.set_label(r"$\hat{\phi}(x,z,\hat t)$")

# # 地面
# ground_line = ax.axhline(ground_z_hat, color="black", linewidth=1)

# # 複数粒子の軌跡と点
# lines_trail = []
# points      = []

# colors = plt.cm.tab10(np.linspace(0, 1, len(labels)))

# for i, lab in enumerate(labels):
#     color = colors[i]
#     # 線と点の色を明示的に同じ color にする
#     line, = ax.plot([], [], "-", linewidth=1, label=lab, color=color)
#     pt,   = ax.plot([], [], "o", markersize=6, color=color)  # 点も同じ色
#     lines_trail.append(line)
#     points.append(pt)

# time_text = ax.text(
#     0.02, 0.95, "", transform=ax.transAxes,
#     va="top", ha="left", color="w"
# )

# ax.set_xlabel(r"$\hat{x}$")
# ax.set_ylabel(r"$\hat{z}$")
# ax.set_title("Regolith transport for multiple friction coefficients")
# ax.legend(loc="upper right", fontsize=8)

# im.set_clim(-1.0, 1.0)

# # =========================================
# # 5. 初期化関数
# # =========================================
# def init():
#     im.set_data(phi_hat_field(t_anim[0]))
#     for line, pt in zip(lines_trail, points):
#         line.set_data([], [])
#         pt.set_data([], [])
#     time_text.set_text("")
#     return [im, *lines_trail, *points, time_text]

# # =========================================
# # 6. フレーム更新関数
# # =========================================
# def update(frame: int):
#     t_now = t_anim[frame]
#     Phi   = phi_hat_field(t_now)
#     im.set_data(Phi)

#     for x_anim, z_anim, line, pt in zip(x_anim_lists, z_anim_lists, lines_trail, points):
#         # 過去の軌跡（周期境界で折りたたむ）
#         x_trail_wrapped = wrap_x_hat(x_anim[:frame+1])
#         line.set_data(x_trail_wrapped, z_anim[:frame+1])

#         # 現在位置（これも [0,2π) に折りたたむ）
#         x_now_wrapped = float(wrap_x_hat(x_anim[frame]))
#         z_now = float(z_anim[frame])
#         pt.set_data([x_now_wrapped], [z_now])

#     time_text.set_text(r"$\hat t = {:.2f}$".format(t_now))

#     return [im, *lines_trail, *points, time_text]



# # =========================================
# # 7. アニメーション生成 & GIF 保存
# # =========================================
# anim = animation.FuncAnimation(
#     fig,
#     update,
#     init_func=init,
#     frames=len(t_anim),
#     interval=60,
#     blit=True,
# )

# plt.close(fig)

# writer = PillowWriter(fps=20)
# output_path = "regolith_mu_compare_1wavelength_accelx.gif"
# anim.save(output_path, writer=writer, dpi=200)

# print(f"Saved animation to {output_path}")


位相差横軸のアニメーション

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter

# =========================================
# 0. データの取り出し ＆ 共通時間軸の作成
# =========================================
t_lists = []
x_lists = []
z_lists = []
labels  = []

for entry in results_by_mu:
    hist_nd = entry["hist_nd"]
    t_arr = np.array(hist_nd["t"])
    x_arr = np.array(hist_nd["x"])
    z_arr = np.array(hist_nd["z"])

    t_lists.append(t_arr)
    x_lists.append(x_arr)
    z_lists.append(z_arr)
    labels.append(entry["p"].label)

# 最小長さに揃える
min_len = min(len(t) for t in t_lists)
t_base  = t_lists[0][:min_len]
x_lists = [x[:min_len] for x in x_lists]
z_lists = [z[:min_len] for z in z_lists]

# =========================================
# 1. フレーム間引き（容量対策）
# =========================================
target_frames = 600
step = max(1, len(t_base) // target_frames)

indices = np.arange(0, len(t_base), step)
t_anim  = t_base[indices]

x_anim_lists = [x[indices] for x in x_lists]
z_anim_lists = [z[indices] for z in z_lists]

# =========================================
# 2. 横軸を「位相 ψ」の 1 波長 [0, 2π) にする
# =========================================
L_psi = 2.0 * np.pi
psi_min = 0.0
psi_max = L_psi

def wrap_psi(psi):
    """位相 ψ を [0, 2π) に折りたたむ"""
    return np.mod(psi, L_psi)

# z 範囲
all_z = np.concatenate(z_anim_lists)
z_min = ground_z_hat
z_data_max = all_z.max()
margin_z = 0.1 * max(z_data_max - z_min, 1.0)
z_max = max(z_data_max + margin_z, 3.0)

# =========================================
# 3. 加速度 a_x(ψ, z) のグリッド（波の上の系なので時間依存なし）
# =========================================
nx, nz = 400, 200
psi_grid = np.linspace(psi_min, psi_max, nx)
z_grid   = np.linspace(z_min, z_max, nz)
PSI, Z   = np.meshgrid(psi_grid, z_grid)

def ax_hat_field():
    """
    波の上に乗った系での x 方向加速度：
    a_x_hat(ψ, z) = -exp(-z) * cos(ψ + φ0)
    （時間依存なし）
    """
    phase = PSI + phi0
    # return -np.exp(-Z) * np.cos(phase)
    return np.exp(-Z) * np.sin(phase)

# =========================================
# 4. 図とアーティストの準備
# =========================================
fig, ax = plt.subplots(figsize=(7, 4))

Ax0 = ax_hat_field()

im = ax.imshow(
    Ax0,
    extent=[psi_min, psi_max, z_min, z_max],
    origin="lower",
    aspect="auto",
    cmap="RdBu_r",
)

cbar = fig.colorbar(im, ax=ax)
cbar.set_label(r"$\hat a_x(\psi,\hat z)$")

# 地面
ground_line = ax.axhline(ground_z_hat, color="black", linewidth=1)

tick_vals   = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
tick_labels = ["0", "π/2", "π", "3π/2", "2π"]
ax.set_xticks(tick_vals)
ax.set_xticklabels(tick_labels)

ax.set_xlabel(r"phase $\psi = \hat x - \Omega \hat t + \phi_0$")
ax.set_ylabel(r"$\hat{z}$")
ax.set_title("Regolith transport in wave frame (multiple friction coefficients)")
ax.legend(loc="upper right", fontsize=8)

im.set_clim(-1.0, 1.0)

# 複数粒子の軌跡と点
lines_trail = []
points      = []

colors = plt.cm.tab10(np.linspace(0, 1, len(labels)))

for i, lab in enumerate(labels):
    color = colors[i]
    line, = ax.plot([], [], "-", linewidth=1, label=lab, color=color)
    pt,   = ax.plot([], [], "o", markersize=6, color=color)
    lines_trail.append(line)
    points.append(pt)

time_text = ax.text(
    0.02, 0.95, "", transform=ax.transAxes,
    va="top", ha="left", color="w"
)

ax.set_xlabel(r"phase $\psi = \hat x - \Omega \hat t + \phi_0$")
ax.set_ylabel(r"$\hat{z}$")
ax.set_title("Regolith transport in wave frame (multiple friction coefficients)")
ax.legend(loc="upper right", fontsize=8)

im.set_clim(-1.0, 1.0)

# =========================================
# 5. 初期化関数
# =========================================
def init():
    # 背景は時間によらず一定
    im.set_data(Ax0)
    for line, pt in zip(lines_trail, points):
        line.set_data([], [])
        pt.set_data([], [])
    time_text.set_text("")
    return [im, *lines_trail, *points, time_text]

# =========================================
# 6. フレーム更新関数
# =========================================
def update(frame: int):
    t_now = t_anim[frame]

    for x_anim, z_anim, line, pt in zip(x_anim_lists, z_anim_lists, lines_trail, points):
        # 位相 ψ = x_hat - Ω t_hat + φ0 を計算して [0,2π) に折りたたむ
        psi_trail = x_anim[:frame+1] - Omega * t_anim[:frame+1] + phi0
        psi_trail_wrapped = wrap_psi(psi_trail)
        line.set_data(psi_trail_wrapped, z_anim[:frame+1])

        psi_now = x_anim[frame] - Omega * t_now + phi0
        psi_now_wrapped = float(wrap_psi(psi_now))
        z_now = float(z_anim[frame])
        pt.set_data([psi_now_wrapped], [z_now])

    time_text.set_text(r"$\hat t = {:.2f}$".format(t_now))

    return [im, *lines_trail, *points, time_text]

# =========================================
# 7. アニメーション生成 & GIF 保存
# =========================================
anim = animation.FuncAnimation(
    fig,
    update,
    init_func=init,
    frames=len(t_anim),
    interval=60,
    blit=True,
)

plt.close(fig)

writer = PillowWriter(fps=20)
output_path = "regolith_mu_compare_wave_frame_x0pi2.gif"
anim.save(output_path, writer=writer, dpi=200)

print(f"Saved animation to {output_path}")


----------------

エネルギーアニメーション（横軸位相差）

In [ ]:
def make_energy_animation_multi(
        max_frames=600,
        outname_prefix="energy_phase_multi"):
    """
    複数の粒子（results_by_mu の全要素）について、
    横軸 ψ = x̂ - Ω t̂ + φ0、縦軸 エネルギー H の
    エネルギー可視化アニメーションを作成する。

    背景: U(ψ) = A sin ψ （A = exp(-R_hat)）
    範囲: ψ ∈ [-π, π]
    粒子: 色分け、点で描画
    H, K の数値表示は "しない"

    前提:
        - results_by_mu が存在する
        - 各 entry["hist_nd"] に "t" と "x" がある
        - Omega, phi0, R_hat が global にある
    """

    # ======================================================
    # 0. 粒子ごとのデータをロード（共通フレーム化）
    # ======================================================

    # 時間配列の共通化（最短に合わせる）
    t_lists = []
    x_lists = []

    for entry in results_by_mu:
        hist = entry["hist_nd"]
        t_arr = np.array(hist["t"])
        x_arr = np.array(hist["x"])
        t_lists.append(t_arr)
        x_lists.append(x_arr)

    min_len = min(len(t) for t in t_lists)
    # 共通時間軸
    t_base = t_lists[0][:min_len]
    # 全粒子の x(t) を同じ長さにそろえる
    x_base_lists = [x[:min_len] for x in x_lists]

    # ------------------------------------------------------
    # フレーム間引き
    # ------------------------------------------------------
    if min_len <= max_frames:
        indices = np.arange(min_len)
    else:
        indices = np.linspace(0, min_len - 1, max_frames, dtype=int)

    t_anim = t_base[indices]
    x_anim_lists = [x[indices] for x in x_base_lists]

    # ======================================================
    # 1. 多粒子の ψ, U, H を計算
    # ======================================================

    # ψ = x̂ - Ω t̂ + φ0 を [-π, π) に折りたたむ関数
    def fold_pi(x):
        return (x + np.pi) % (2*np.pi) - np.pi

    # ポテンシャル振幅 A = exp(-R_hat)
    A = np.exp(-R_hat)

    # 全粒子の ψ, U, H を格納
    psi_lists = []
    U_lists   = []
    H_lists   = []

    for x_anim in x_anim_lists:
        # 速度（ẋ）を数値微分
        vx_anim = np.gradient(x_anim, t_anim)

        psi_raw = x_anim - Omega * t_anim + phi0
        psi = fold_pi(psi_raw)

        # U(ψ)
        U = A * np.sin(psi)

        # H = 1/2 (ψ')^2 + U, ψ' = ẋ - Ω
        psi_dot = vx_anim - Omega
        H = 0.5 * psi_dot**2 + U

        psi_lists.append(psi)
        U_lists.append(U)
        H_lists.append(H)

    # ======================================================
    # 2. 図の準備（背景ポテンシャル U(ψ)）
    # ======================================================
    fig, ax = plt.subplots(figsize=(7, 4))

    # 背景 U(ψ) = A sin ψ
    psi_grid = np.linspace(-np.pi, np.pi, 400)
    U_grid = A * np.sin(psi_grid)
    ax.plot(psi_grid, U_grid, color="gray", linewidth=2,
            label=r"Background $U(\psi)=A\sin\psi$")

    # 目盛り設定（−π, −π/2, 0, π/2, π）
    ticks = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi]
    tick_labels = [r"$-\pi$", r"$-\pi/2$", r"$0$", r"$\pi/2$", r"$\pi$"]
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)

    # 粒子ごとの色情報
    colors = plt.cm.tab10(np.linspace(0, 1, len(results_by_mu)))

    # 各粒子の点
    point_artists = []
    for color, entry in zip(colors, results_by_mu):
        pt, = ax.plot([], [], "o", color=color, markersize=7,
                      label=entry["p"].label)
        point_artists.append(pt)

    # 時間表示
    time_text = ax.text(
        0.02, 0.95, "", transform=ax.transAxes,
        va="top", ha="left"
    )

    ax.set_xlabel(r"Phase $\psi$")
    ax.set_ylabel("Energy")
    ax.set_title(f"Energy-phase animation (Omega={Omega})")
    ax.legend(loc="upper right", fontsize=8)

    # y軸範囲（ポテンシャル全体が必ず見えるように）
    all_H = np.concatenate(H_lists)
    amp = abs(np.ptp(U_grid))  # = U_grid.max() - U_grid.min()

    # ポテンシャルの全振幅を必ず含める
    base_min = U_grid.min()
    base_max = U_grid.max()

    # H がポテンシャルより上に来る場合に備えて少し上に余裕を持たせる
    y_min = base_min - 0.2 * amp
    y_max = max(base_max, all_H.max()) + 0.3 * amp

    ax.set_ylim(y_min, y_max)

    # ======================================================
    # 3. アニメーション関数
    # ======================================================
    def init():
        for pt in point_artists:
            pt.set_data([], [])
        time_text.set_text("")
        return point_artists + [time_text]

    def update(frame):
        t_now = t_anim[frame]

        # 粒子を1つずつ更新
        for pt, psi_arr, U_arr in zip(point_artists, psi_lists, U_lists):
            psi_now = psi_arr[frame]
            U_now   = U_arr[frame]
            pt.set_data([psi_now], [U_now])

        time_text.set_text(f"$\\hat t$ = {t_now:.2f}")

        return point_artists + [time_text]

    # ======================================================
    # 4. アニメ保存
    # ======================================================
    anim = animation.FuncAnimation(
        fig, update, init_func=init,
        frames=len(t_anim), interval=60, blit=True
    )

    plt.close(fig)

    outname = f"{outname_prefix}_Omega{Omega:.2f}.gif"
    writer = PillowWriter(fps=20)
    anim.save(outname, writer=writer, dpi=200)

    print(f"Saved energy-phase animation to {outname}")


In [ ]:
make_energy_animation_multi()

エネルギー位相差可視化new

In [ ]:
def compute_energy_series(target_idx=0, max_frames=600, v_eps=1e-6):
    """
    results_by_mu[target_idx] から
      t, ψ, U, K, H, ラベル, A, is_static
    を計算して返すヘルパー。

    is_static: そのフレームで「静止中（|vx| < v_eps かつ接触中）」かどうか
    """

    entry   = results_by_mu[target_idx]
    hist_nd = entry["hist_nd"]
    label   = entry["p"].label

    # フル解像度の時間・位置・速度
    t_full  = np.array(hist_nd["t"])
    x_full  = np.array(hist_nd["x"])
    vx_full = np.array(hist_nd["vx"])

    # 接触フラグがあれば利用
    if "in_contact" in hist_nd:
        in_contact_full = np.array(hist_nd["in_contact"], dtype=bool)
    else:
        in_contact_full = np.ones_like(vx_full, dtype=bool)

    # フレーム数を間引き
    n_full = len(t_full)
    if n_full <= max_frames:
        indices = np.arange(n_full)
    else:
        indices = np.linspace(0, n_full - 1, max_frames, dtype=int)

    t   = t_full[indices]
    x   = x_full[indices]
    vx  = vx_full[indices]
    in_contact = in_contact_full[indices]

    # 静止判定（vx が十分小さく、かつ接触）
    is_static = (np.abs(vx) < v_eps) & in_contact

    # 位相 ψ = x̂ - Ω t̂ + φ0 を [-π, π) に折りたたみ
    psi_raw = x - Omega * t + phi0
    psi     = (psi_raw + np.pi) % (2.0 * np.pi) - np.pi

    # 相対速度 ψ' = ẋ - Ω
    psi_dot = vx - Omega

    # ポテンシャル振幅 A （z ≃ R_hat で切った1Dモデル）
    A = np.exp(-R_hat)

    # エネルギー
    U = A * np.sin(psi)         # ポテンシャル
    K = 0.5 * psi_dot**2        # 運動エネルギー
    H = K + U                   # 全エネルギー

    return t, psi, U, K, H, label, A, is_static


In [ ]:
def make_energy_animation(target_idx=0,
                          max_frames=600,
                          outname_prefix="energy_phase"):

    # ここで t, ψ, U, K, H, is_static を一括取得
    t, psi, U, K, H, label, A, is_static = compute_energy_series(
        target_idx=target_idx,
        max_frames=max_frames
    )

    # ===============================
    # 図の準備（横: ψ, 縦: エネルギー）
    # ===============================
    fig_E, ax_E = plt.subplots(figsize=(6, 4))
    fig_E.subplots_adjust(bottom=0.22) 

    # 背景ポテンシャル U(ψ)
    psi_grid = np.linspace(-np.pi, np.pi, 400)
    U_grid = A * np.sin(psi_grid)
    ax_E.plot(psi_grid, U_grid, label=r"$U(\psi)=A\sin\psi$")

    # 粒子位置（点）: デフォルト色を「動いているときの色」にしておく
    particle_point_E, = ax_E.plot(
        [], [], "o", markersize=8, label="particle", color="C0"
    )

    # 総エネルギー H の水平線
    energy_line_E, = ax_E.plot(
        psi_grid,
        np.zeros_like(psi_grid),
        "--",
        linewidth=1.5,
        label="total energy $H$",
    )

    # 運動エネルギーを表す縦線（U から H まで）
    kinetic_line_E, = ax_E.plot([], [], "-", linewidth=2, label="kinetic energy")

    # 情報テキスト
    text_info_E = ax_E.text(
        0.02, 0.95, "", transform=ax_E.transAxes,
        va="top", ha="left"
    )

    ax_E.set_xlabel(r"phase $\psi = \hat x - \Omega \hat t + \phi_0$")
    ax_E.set_ylabel("energy ")
    ax_E.legend(loc="upper right", fontsize=8)

    ax_E.set_xlim(-np.pi, np.pi)
    ticks = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi]
    tick_labels = [r"$-\pi$", r"$-\pi/2$", r"$0$", r"$\pi/2$", r"$\pi$"]
    ax_E.set_xticks(ticks)
    ax_E.set_xticklabels(tick_labels)

    amp      = abs(np.ptp(U_grid))
    base_min = U_grid.min()
    base_max = U_grid.max()
    y_min = base_min - 0.2 * amp
    y_max = max(base_max, H.max()) + 0.3 * amp
    ax_E.set_ylim(y_min, y_max)

    # a_x > 0 の領域（左右の耳）
    ax_E.axvspan(-np.pi, -np.pi/2, alpha=0.08, color="C0")
    ax_E.axvspan( np.pi/2,  np.pi, alpha=0.08, color="C0")
    # a_x < 0 の領域（真ん中）
    ax_E.axvspan(-np.pi/2,  np.pi/2, alpha=0.08, color="C1")

    y_bot  = y_min + 0.08*(y_max - y_min)
    ax_E.text(-3*np.pi/4, y_bot,  r"$a_x>0$", ha="center", va="bottom")
    ax_E.text( 3*np.pi/4, y_bot,  r"$a_x>0$", ha="center", va="bottom")
    ax_E.text( 0,         y_bot,  r"$a_x<0$", ha="center", va="bottom")

    # ===============================
    # アニメーション関数
    # ===============================
    def init_energy():
        particle_point_E.set_data([], [])
        kinetic_line_E.set_data([], [])
        energy_line_E.set_ydata(np.zeros_like(psi_grid))
        text_info_E.set_text("")
        return particle_point_E, kinetic_line_E, energy_line_E, text_info_E

    def update_energy(frame: int):
        t_now   = t[frame]
        psi_now = psi[frame]
        U_now   = U[frame]
        H_now   = H[frame]
        K_now   = K[frame]

        # 粒子位置（ポテンシャル曲線上）
        particle_point_E.set_data([psi_now], [U_now])

        # ★ 静止中かどうかで色を切り替え ★
        if is_static[frame]:
            particle_point_E.set_color("C3")   # 例: 静止中はオレンジ/赤系
        else:
            particle_point_E.set_color("C0")   # 動いているときの色

        # 総エネルギー H の水平線
        energy_line_E.set_ydata(H_now * np.ones_like(psi_grid))

        # 運動エネルギー: U から H までの縦棒
        kinetic_line_E.set_data([psi_now, psi_now], [U_now, H_now])

        # テキスト
        text_info_E.set_text(
            f"$\\hat t$ = {t_now:.2f}\n"
            f"H = {H_now:.3f}\n"
            f"K = {K_now:.3f}"
        )

        return particle_point_E, kinetic_line_E, energy_line_E, text_info_E

    anim_E = animation.FuncAnimation(
        fig_E,
        update_energy,
        init_func=init_energy,
        frames=len(t),
        interval=60,
        blit=True,
    )

    plt.close(fig_E)

    outname = f"{outname_prefix}_mu{target_idx}_x0pi2.gif"
    writer_E = PillowWriter(fps=20)
    anim_E.save(outname, writer=writer_E, dpi=200)

    print(f"Saved energy-phase animation to {outname}")


In [ ]:
def plot_energy_time(target_idx=0,
                     max_frames=600,
                     show_UK=True):
    """
    compute_energy_series() で使っている
    H, U, K をそのまま時間に対してプロットする。

    ・横軸: t_hat
    ・縦軸: H(t) （オプションで U(t), K(t) も）
    ・しきい値 H = A の水平線も描画
    """

    t, psi, U, K, H, label, A, _ = compute_energy_series(
        target_idx=target_idx,
        max_frames=max_frames
    )

    fig, ax = plt.subplots(figsize=(7, 4))

    ax.plot(t, H, label="H(t)", linewidth=2)

    if show_UK:
        ax.plot(t, U, "--", label="U(t)", linewidth=1)
        ax.plot(t, K, ":",  label="K(t)", linewidth=1)

    ax.axhline(A, color="gray", linestyle="--", linewidth=1,
               label="H = A (trap threshold)")

    ax.set_xlabel(r"$\hat t$")
    ax.set_ylabel("energy (nondimensional)")
    ax.set_title(f"Energy vs Time  ({label})")
    ax.legend(loc="best", fontsize=8)
    ax.grid(True)
    fig.tight_layout()

    plt.show()
    return fig, ax


In [ ]:
make_energy_animation(target_idx=0)


In [ ]:
make_energy_animation(target_idx=1)
make_energy_animation(target_idx=2)

In [ ]:
# 同じ粒子について H(t) を表示
plot_energy_time(target_idx=0)
plot_energy_time(target_idx=1)
# plot_energy_time(target_idx=5)

-------------------------------

静止から動摩擦への切り替え点の探索

In [ ]:
def find_static_to_kinetic_events(mu_s_target, mu_k_target, v_eps=1e-6):
    """
    指定した (μs, μk) のケースについて，
    粒子が「停止状態（|v_x| < v_eps）→動き出す（|v_x| >= v_eps）」に
    切り替わる時刻 t_hat と位置 x_hat を列挙する。

    v_eps : 「停止」とみなす速度のしきい値（無次元）
    """
    # 対応する結果を探す
    for entry in results_by_mu:
        if np.isclose(entry["mu_s"], mu_s_target) and np.isclose(entry["mu_k"], mu_k_target):
            hist_nd = entry["hist_nd"]
            break
    else:
        print(f"結果が見つかりません: μs={mu_s_target}, μk={mu_k_target}")
        return []

    t_hat = np.array(hist_nd["t"])
    x_hat = np.array(hist_nd["x"])
    vx_hat = np.array(hist_nd["vx"])

    # （もし接触フラグを保存しているなら併用する）
    if "in_contact" in hist_nd:
        in_contact = np.array(hist_nd["in_contact"], dtype=bool)
    else:
        # 情報がなければ「常に接触している」とみなす
        in_contact = np.ones_like(vx_hat, dtype=bool)

    # 停止状態かどうか（静止摩擦が効いている候補）
    is_static = (np.abs(vx_hat) < v_eps) & in_contact

    # 1ステップ前は static, 今ステップは non-static になったインデックス
    #   is_static[i-1] = True かつ is_static[i] = False
    idxs = np.where(is_static[:-1] & (~is_static[1:]))[0] + 1

    events = []
    for idx in idxs:
        t_sw = t_hat[idx]
        x_sw = x_hat[idx]
        events.append((t_sw, x_sw))
        print(f"static→kinetic at index {idx}:  t_hat = {t_sw:.6g},  x_hat = {x_sw:.6g}")

    return events


In [ ]:
events = find_static_to_kinetic_events(mu_s_target=1.0, mu_k_target=0.2, v_eps=1e-5)


In [ ]:
events = find_static_to_kinetic_events(mu_s_target=1.0, mu_k_target=0.2, v_eps=1e-6)
